# Práctica Final: Clasificación con Scikit-learn y MLflow

En esta práctica, utilizarás un conjunto de datos de Scikit-learn (podeís usar el mismo que en el notebook de Intro MLFlow) para entrenar un modelo de clasificación.

Pasos a seguir: 

    Exploración de Datos: Analiza el conjunto de datos proporcionado para comprender su estructura y contenido.

    Preprocesamiento de Texto: Realiza tareas de preprocesamiento de texto, como tokenización y vectorización, para preparar los datos para el modelado.

    Entrenamiento del Modelo: Utiliza algoritmos de clasificación de Scikit-learn para entrenar un modelo con los datos preprocesados.

    Evaluación del Modelo: Evalúa el rendimiento del modelo utilizando métricas de evaluación estándar como precisión y recall.

    Registro de Métricas con MLflow: Utiliza MLflow para registrar métricas y hiperparámetros durante el entrenamiento, facilitando la gestión y comparación de experimentos.


Nota: Dado que no voy a poder tener acceso a vuestros logs de MLFlow añadirme las imagenes de la interfaz de MLFlow en el notebook

# 🧠 Clasificador de texto
🔍 ¿Qué hace este clasificador?
Este clasificador tiene como objetivo:

Predecir la categoría o clase de un texto, basándose en patrones aprendidos durante el entrenamiento.

En este caso, ha sido entrenado con textos pertenecientes a diferentes temas (como religión, ciencia médica, computación gráfica o el espacio), por lo que puede identificar automáticamente a qué categoría pertenece un nuevo texto, dependiendo del modelo que se escoja y la categoría usada durante el entrenamiento.



![Init](screenshots/log-init.png)

![Init](screenshots/log.png)

![Init](screenshots/comparation-8v.png)

![Init](screenshots/models.png)

![Init](screenshots/model_versions.png)

![Init](screenshots/comparation_versions.png)
Podemos que ver la versión 3 es la que mejor precisión tiene, si en caso necestaramos usarlo en producción usaria: model_uri = f"models:/{model_name}/3" 

![Init](screenshots/comparation_version_visual.png)

## Generar .py de funciones y main con al menos dos argumentos de entrada.

In [1]:
import argparse
import mlflow
import mlflow.sklearn

from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [3]:
def calculate_metrics(y_true, y_pred):
    return {
        "accuracy": accuracy_score(y_true, y_pred),
        "precision": precision_score(y_true, y_pred),
        "recall": recall_score(y_true, y_pred),
        "f1_score": f1_score(y_true, y_pred),
    }

In [4]:
def create_pipeline(
    max_iter=100,
    C=1.0,
    max_features=None,
    ngram_max=1,
):
    pipeline = Pipeline(
        [
            (
                "vectorizer",
                TfidfVectorizer(
                    max_features=max_features,
                    ngram_range=(1, ngram_max),
                ),
            ),
            (
                "clf",
                LogisticRegression(
                    max_iter=max_iter,
                    C=C,
                ),
            ),
        ]
    )
    return pipeline

In [5]:
def sanitize_name(name):
    return name.replace(".", "_").replace("/", "_").replace(" ", "_")

In [7]:
def main(
    categoria1,
    categoria2,
    max_iter,
    C,
    max_features,
    ngram_max,
):
    data = fetch_20newsgroups(
        subset="all",
        categories=[categoria1, categoria2],
        remove=("headers", "footers", "quotes"),
    )
    X_train, X_test, y_train, y_test = train_test_split(
        data.data, data.target, test_size=0.2, random_state=42
    )

    pipeline = create_pipeline(
        max_iter=max_iter,
        C=C,
        max_features=max_features,
        ngram_max=ngram_max,
    )

    mlflow.set_experiment("text_classification")
    with mlflow.start_run(run_name=f"{categoria1}_vs_{categoria2}"):
        pipeline.fit(X_train, y_train)
        y_pred = pipeline.predict(X_test)

        metricas = calculate_metrics(y_test, y_pred)
        for clave, valor in metricas.items():
            mlflow.log_metric(clave, valor)

        # Loguear hiperparámetros
        mlflow.log_param("max_iter", max_iter)
        mlflow.log_param("C", C)
        mlflow.log_param("max_features", max_features)
        mlflow.log_param("ngram_max", ngram_max)

        mlflow.log_param("modelo", f"LogisticRegression_{categoria1}_vs_{categoria2}")
        mlflow.log_param("vectorizer", f"TfidfVectorizer_{categoria1}_vs_{categoria2}")

        modelo_name = f"modelo_texto_{categoria1}_vs_{categoria2}"
        modelo_name = sanitize_name(modelo_name)

        registered_name = f"TextoClasificador_{categoria1}_vs_{categoria2}"
        registered_name = sanitize_name(registered_name)

        mlflow.sklearn.log_model(
            pipeline, modelo_name, registered_model_name=registered_name
        )

        print(f"\nResultado entre '{categoria1}' y '{categoria2}':")
        for k, v in metricas.items():
            print(f"{k}: {v:.3f}")

In [ ]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--categoria1",
        required=True,
        help="Nombre de la primera categoría (ej: 'sci.space')",
    )
    parser.add_argument(
        "--categoria2",
        required=True,
        help="Nombre de la segunda categoría (ej: 'comp.graphics')",
    )
    parser.add_argument(
        "--max_iter",
        type=int,
        default=100,
        help="Número máximo de iteraciones para LogisticRegression",
    )
    parser.add_argument(
        "--C",
        type=float,
        default=1.0,
        help="Parámetro de regularización para LogisticRegression",
    )
    parser.add_argument(
        "--max_features",
        type=int,
        default=None,
        help="Máximo número de características para TfidfVectorizer",
    )
    parser.add_argument(
        "--ngram_max", type=int, default=1, help="Máximo ngram para TfidfVectorizer"
    )

    args = parser.parse_args()

    main(
        args.categoria1,
        args.categoria2,
        args.max_iter,
        args.C,
        args.max_features,
        args.ngram_max,
    )

## Comando para desplegar el modelo

In [10]:
!py mlflow_load.py --categoria1 sci.space --categoria2 comp.graphics --max_iter 200 --C 0.5 --max_features 10000 --ngram_max 2


Resultado entre 'sci.space' y 'comp.graphics':
accuracy: 0.898
precision: 0.917
recall: 0.880
f1_score: 0.898


2025/06/15 18:07:48 INFO mlflow.tracking.fluent: Experiment with name 'text_classification' does not exist. Creating a new experiment.
2025/06/15 18:07:52 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/06/15 18:08:02 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'TextoClasificador_sci_space_vs_comp_graphics'.
Created version '1' of model 'TextoClasificador_sci_space_vs_comp_graphics'.


## Comando para realizar predicción

In [14]:
!py mlflow_predict.py "NASA launched a new satellite this month"

Texto: NASA launched a new satellite this month
Predicción (label numérico): 1
Categoría predicha: sci.space


## Práctica parte FastAPI

### Para esta parte de la práctica teneis que generar un script con al menos 5 modulos app.get y dos de ellos tienen que ser pipelines de HF. 

### Parte de la practica se tendra que entregar en capturas de pantalla. Las capturas de pantalla a adjuntas son las siguientes. 

### 1. Captura de la pantalla docs con al menos 5 modulos. 
### 2. Captura de cada una de los modulos con la respuesta dentro de docs. 
### 3. Captura de cada uno de los modulos en la llamada https.
### 4. Todo el codigo usado durante el proceso. Notebooks y scripts.

### Opcional

### 5. Despliegue del script en GCP Cloud Run

![Init](screenshots/api.png)

![Init](screenshots/root.png)

![Init](screenshots/greet.png)

![Init](screenshots/double.png)

## Analizador de sentimientos (HugginFace)
![Init](screenshots/sentyments.png)

## Traducir texto de Inglés a Frances (HugginFace)
![Init](screenshots/translate.png)

## Generación de imagenes (HugginFace)
![Init](screenshots/generate_image.png)
![Init](screenshots/image_base64.png)

## Extraer embedding de un texto (HugginFace)
![Init](screenshots/embed.png)

## Predecir categoria (space o computer graphics) de texto de la práctica MLFLOW
![Init](screenshots/predict_text_category.png)

In [17]:
!uvicorn api_fast:app --reload

^C


# Código completo aquí o api_fast.py

In [ ]:
from fastapi import FastAPI
from transformers import pipeline, AutoTokenizer, AutoModel
from diffusers import StableDiffusionPipeline
import torch
import base64
from io import BytesIO
import mlflow
from sklearn.datasets import fetch_20newsgroups

app = FastAPI(title="API")

# === MODELOS DE TRANSFORMERS ===
sentiment_analyzer = pipeline("sentiment-analysis")
translator = pipeline("translation_en_to_fr")

# === MODELO DE EMBEDDINGS ===
tokenizer_embed = AutoTokenizer.from_pretrained("Qwen/Qwen3-Embedding-0.6B")
model_embed = AutoModel.from_pretrained("Qwen/Qwen3-Embedding-0.6B")

# === MODELO DE IMAGEN (Stable Diffusion) ===
dreamlike_pipe = StableDiffusionPipeline.from_pretrained(
    "dreamlike-art/dreamlike-anime-1.0", torch_dtype=torch.float16, use_safetensors=True
).to("cuda")


@app.get("/")
def read_root():
    return {"message": "Welcome to the AI API!"}


@app.get("/greet")
def greet(name: str = "user"):
    return {"message": f"Hello, {name}!"}


@app.get("/double")
def double(number: int):
    return {"result": number * 2}


@app.get("/sentiment_analysis")
def sentiment_analysis(text: str):
    result = sentiment_analyzer(text)
    return {"sentiment": result}


@app.get("/translate")
def translate(text: str):
    result = translator(text)
    return {"translation": result[0]["translation_text"]}


@app.get("/generate_image")
def generate_image(prompt: str = "an anime girl in a magical forest"):
    image = dreamlike_pipe(prompt).images[0]

    buffer = BytesIO()
    image.save(buffer, format="PNG")
    img_base64 = base64.b64encode(buffer.getvalue()).decode("utf-8")

    return {"prompt": prompt, "image_base64": img_base64}


@app.get("/embed_text")
def embed_text(text: str):
    inputs = tokenizer_embed(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model_embed(**inputs)
        embeddings = outputs.last_hidden_state[:, 0, :]  # CLS token
    return {"text": text, "embedding": embeddings[0].tolist()}


# === FUNCIONES MLflow ===


def sanitize_name(name):
    return name.replace(".", "_").replace("/", "_").replace(" ", "_")


@app.get("/predict_text_category")
def predict_text_category(text: str = "NASA launched a new satellite this month"):
    # Define categorías
    categoria1 = "sci.space"
    categoria2 = "comp.graphics"
    categorias = [categoria1, categoria2]

    # Obtener los nombres de clases
    target_names = fetch_20newsgroups(
        subset="train", categories=categorias
    ).target_names

    # Modelo en MLflow
    model_name = f"TextoClasificador_{categoria1}_vs_{categoria2}"
    model_name = sanitize_name(model_name)
    model_uri = f"models:/{model_name}/latest"

    try:
        model = mlflow.pyfunc.load_model(model_uri)
        prediccion = model.predict([text])
        clase_predicha = target_names[prediccion[0]]

        return {
            "text": text,
            "predicted_label": int(prediccion[0]),
            "predicted_category": clase_predicha,
        }

    except Exception as e:
        return {"error": str(e)}
